In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import re


In [6]:
data = {
    'id': [1, 2, 2, 4, 5],  # ID 2 está duplicado
    'nombre': ['Ana', 'Luis', 'Pedro', None, 'Laura'],  # Faltan datos en 'nombre'
    'edad': [23, -45, 35, 22, 29],  # Edad negativa, lo cual es inválido
    'correo': ['ana@example.com', 'luis@example', 'pedro@example.com', 'marta@.com', 'laura@example.com'],  # Correos inválidos
    'fecha_registro': ['2024-01-10', '2025-12-05', '2024-06-15', '2023-12-01', None]  # Faltan fechas de registro
}

df = pd.DataFrame(data)

In [7]:
df['fecha_registro'] = pd.to_datetime(df['fecha_registro'], errors='coerce')

print("Conjunto de Datos Inicial:\n", df)

Conjunto de Datos Inicial:
    id nombre  edad             correo fecha_registro
0   1    Ana    23    ana@example.com     2024-01-10
1   2   Luis   -45       luis@example     2025-12-05
2   2  Pedro    35  pedro@example.com     2024-06-15
3   4   None    22         marta@.com     2023-12-01
4   5  Laura    29  laura@example.com            NaT


#Precisión (Accuracy):
Los datos precisos son aquellos que representan correctamente la realidad.
En este caso, verificaremos la precisión de la columna edad, asegurándonos de que todas las edades sean positivas

In [8]:
df['edad_precisa'] = df['edad'].apply(lambda x: x > 0 if pd.notnull(x) else False)
print("\nPrecisión (Edad positiva):\n", df[['id', 'edad', 'edad_precisa']])


Precisión (Edad positiva):
    id  edad  edad_precisa
0   1    23          True
1   2   -45         False
2   2    35          True
3   4    22          True
4   5    29          True


#Integridad (Completeness):
La integridad se refiere a la ausencia de valores faltantes. Vamos a verificar si existen valores nulos en las columnas críticas, como nombre y fecha_registro.

In [9]:
df['nombre_completo'] = df['nombre'].notna()
df['fecha_completa'] = df['fecha_registro'].notna()
print("\nIntegridad (Valores completos en 'nombre' y 'fecha_registro'):\n", df[['id', 'nombre_completo', 'fecha_completa']])


Integridad (Valores completos en 'nombre' y 'fecha_registro'):
    id  nombre_completo  fecha_completa
0   1             True            True
1   2             True            True
2   2             True            True
3   4            False            True
4   5             True           False


#Validez (Validity):
La validez se refiere a si los datos cumplen con las reglas o estándares. Aquí, verificaremos si los correos electrónicos son válidos utilizando una expresión regular.

In [10]:
def es_correo_valido(correo):
    regex = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    if pd.notna(correo):
        return bool(re.match(regex, correo))
    return False

df['correo_valido'] = df['correo'].apply(es_correo_valido)
print("\nValidez (Correos electrónicos válidos):\n", df[['id', 'correo', 'correo_valido']])


Validez (Correos electrónicos válidos):
    id             correo  correo_valido
0   1    ana@example.com           True
1   2       luis@example          False
2   2  pedro@example.com           True
3   4         marta@.com          False
4   5  laura@example.com           True


#Coherencia (Consistency):
La coherencia implica que los datos sean uniformes a través del tiempo o en diferentes conjuntos. En este ejemplo, podemos verificar la coherencia en fecha_registro asegurándonos de que las fechas no sean futuras.

In [11]:
hoy = datetime.now()
df['fecha_coherente'] = df['fecha_registro'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') <= hoy if pd.notnull(x) else False)
print("\nCoherencia (Fechas de registro no futuras):\n", df[['id', 'fecha_registro', 'fecha_coherente']])


Coherencia (Fechas de registro no futuras):
    id fecha_registro  fecha_coherente
0   1     2024-01-10             True
1   2     2025-12-05            False
2   2     2024-06-15             True
3   4     2023-12-01             True
4   5            NaT            False


#Unicidad (Uniqueness):
La unicidad se refiere a la ausencia de duplicados. Aquí, verificaremos si la columna id tiene valores duplicados.

In [12]:
df['id_unico'] = ~df['id'].duplicated(keep=False)
print("\nUnicidad (ID único):\n", df[['id', 'id_unico']])


Unicidad (ID único):
    id  id_unico
0   1      True
1   2     False
2   2     False
3   4      True
4   5      True


#Oportunidad (Timeliness):
La oportunidad se refiere a la actualidad de los datos. Por ejemplo, podríamos verificar si fecha_registro está dentro de un rango de tiempo relevante.

In [13]:
df['fecha_oportuna'] = df['fecha_registro'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') >= datetime(2022, 1, 1) if pd.notnull(x) else False)
print("\nOportunidad (Fechas de registro recientes):\n", df[['id', 'fecha_registro', 'fecha_oportuna']])


Oportunidad (Fechas de registro recientes):
    id fecha_registro  fecha_oportuna
0   1     2024-01-10            True
1   2     2025-12-05            True
2   2     2024-06-15            True
3   4     2023-12-01            True
4   5            NaT           False


#Aptitud (Fitness):
La aptitud evalúa si los datos son útiles para el propósito previsto. Aquí, determinaremos si todos los campos necesarios para un propósito específico (por ejemplo, contactar a un cliente) están presentes y son válidos.

In [14]:
df['datos_aptos'] = df.apply(lambda row: row['nombre_completo'] and row['correo_valido'] and row['fecha_completa'], axis=1)
print("\nAptitud (Datos aptos para contacto):\n", df[['id', 'datos_aptos']])


Aptitud (Datos aptos para contacto):
    id  datos_aptos
0   1         True
1   2        False
2   2         True
3   4        False
4   5        False


#Ejemplo "Real"

In [29]:
df = pd.read_csv("tips.csv")
print(df.head())


   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [30]:
print(df.describe())

       total_bill         tip        size
count  244.000000  244.000000  244.000000
mean    19.785943    2.998279    2.569672
std      8.902412    1.383638    0.951100
min      3.070000    1.000000    1.000000
25%     13.347500    2.000000    2.000000
50%     17.795000    2.900000    2.000000
75%     24.127500    3.562500    3.000000
max     50.810000   10.000000    6.000000


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   total_bill  244 non-null    float64
 1   tip         244 non-null    float64
 2   sex         244 non-null    object 
 3   smoker      244 non-null    object 
 4   day         244 non-null    object 
 5   time        244 non-null    object 
 6   size        244 non-null    int64  
 7   tipCom      244 non-null    bool   
 8   sexCom      244 non-null    bool   
 9   sized       244 non-null    bool   
dtypes: bool(3), float64(2), int64(1), object(4)
memory usage: 14.2+ KB


In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro




In [32]:
def norma(var):
	# Realizar la prueba de Shapiro-Wilk para normalidad
	stat, p_value = shapiro(df[var])
	print(f"Estadístico de Shapiro-Wilk: {stat:.4f}, Valor p: {p_value:.4f}")

	# Interpretar el resultado
	alpha = 0.05
	if p_value > alpha:
		print("No se rechaza la hipótesis nula. Los datos siguen una distribución normal (al nivel de significancia del 5%).")
	else:
		print("Se rechaza la hipótesis nula. Los datos no siguen una distribución normal (al nivel de significancia del 5%).")

In [52]:
#total_bill         tip        size
norma('size')

Estadístico de Shapiro-Wilk: 0.7120, Valor p: 0.0000
Se rechaza la hipótesis nula. Los datos no siguen una distribución normal (al nivel de significancia del 5%).
